In [59]:
import os
import pandas as pd
import json
import sys
import shortuuid
import numpy
import datetime

In [90]:
#read multiple csv and xlsx files collected

#data collected from International sources
df1 = pd.read_csv('data/dataset_1.csv')
df1['Last Retrieved Time'] = pd.to_datetime(df1['Last Retrieved Time'])
df1 = df1.fillna(method='ffill')
print(df1.shape)

#data collected from karnataka government website
df2 = pd.read_excel('data/dataset_2.xlsx')
df2.rename(columns={"Website" : "Source URL", "Question" : "question", "Answers" : "text"}, inplace=True)
df2['title']  = 'Info from Government'
df2['Organization']  = 'Govt of Karnataka'
df2['Country'] = 'India'
df2['Last Retrieved Time'] = datetime. datetime. today()
df2['context'] = df2['text']
df2.drop(columns=['Unnamed: 3'], inplace=True)
df2 = df2.fillna(method='ffill')
print(df2.shape)

#data collected from WHO website
df3 = pd.read_csv('data/dataset_3.csv')
df3['Last Retrieved Time'] = pd.to_datetime(df3['Last Retrieved Time'])
df3 = df3.fillna(method='ffill')
print(df3.shape)

df = pd.concat([df1, df2, df3], sort=False)
print("Final dataset size : ",df.shape)
print(df.sample(5))

(90, 9)
(277, 9)
(41, 9)
Final dataset size :  (408, 9)
                    title                                           question  \
31                General                       Is covid-19 the same as sars   
39   Info from Government  Are face masks effective in protecting against...   
37                General  Can humans become infected with the COVID-19 f...   
216  Info from Government        May tourists continue to travel to Belgium?   
32                General             Should I wear a mask to protect myself   

                                               context  \
31   No. The virus that causes COVID-19 and the one...   
39   If you are infected, the use of surgical face ...   
37   Coronaviruses are a large family of viruses th...   
216  No, tourism is forbidden. All recreational act...   
32   Only wear a mask if you are ill with COVID-19 ...   

                                                  text  \
31                                                  No  

In [ ]:
print(df[]

In [70]:
# convert the data in squavd format to be fed to bert based algorithms

def f(a,b):
    return(str(a).find(str(b)))

df['answer_start'] = df[['context','text']].apply(lambda x: f(*x), axis=1)
df['id'] = df['question'].apply(lambda x: shortuuid.uuid(name=x))
df.to_csv('data/dataset_collected.csv', index=False)
df.head()

,title,question,context,text,Source,Source URL,Organization,Country,Last Retrieved Time,answer_start,id
0,General,What is a coronavirus,Coronaviruses are a large family of viruses wh...,Coronaviruses are a large family of viruses wh...,Website,https://www.who.int/news-room/q-a-detail/q-a-c...,WHO,International,2020-03-20 11:43:00,0,VrNUodbUMYQdVpZBVeMtY7
1,General,What kind of diseases are caused by corona virus,Coronaviruses are a large family of viruses wh...,coronaviruses are known to cause respiratory i...,Website,https://www.who.int/news-room/q-a-detail/q-a-c...,WHO,International,2020-03-20 11:43:00,110,GmGhdcFwDK3UxDwxTCPqBy
2,General,What is covid-19,COVID-19 is the infectious disease caused by t...,COVID-19 is the infectious disease caused by t...,Website,https://www.who.int/news-room/q-a-detail/q-a-c...,WHO,International,2020-03-20 11:43:00,0,eMj3qhjizrNy6KXDG7PzLx
3,General,When does corona disease started,COVID-19 is the infectious disease caused by t...,"Wuhan, China, in December 2019.",Website,https://www.who.int/news-room/q-a-detail/q-a-c...,WHO,International,2020-03-20 11:43:00,156,KWCAH7Wqdk757HhaoSQVGV
4,General,what are the symptoms of covid-19,The most common symptoms of COVID-19 are fever...,The most common symptoms of COVID-19 are fever...,Website,https://www.who.int/news-room/q-a-detail/q-a-c...,WHO,International,2020-03-20 11:43:00,0,8uJkPD3CFEtnCubUKP5nVC


In [71]:
spec_chars = ["!",'"',"#","%","&","'","(",")",
              "*","+",",","-",".","/",":",";","<",
              "=",">","?","@","[","\\","]","^","_",
              "`","{","|","}","~","–"]
for char in spec_chars:
    df['context'] = df['context'].str.replace(char, ' ')
    df['text'] = df['text'].str.replace(char, ' ')
    
df['context'] = df['context'].str.split().str.join(" ")
df['text'] = df['text'].str.split().str.join(" ")
df['title'] = df['title'].str.split().str.join(" ")

In [72]:
def to_dictionary(df):
   
    grouped1 = df.groupby(['title'])
    dictList1 = []
    for key1, value1 in grouped1:
        anotherDict1 = {}
        
                                
        dictList2 = []
        anotherDict2 = {}
        title = key1
        anotherDict1['title'] = title
        grouped2 = df.loc[df['title'] == title].groupby(['context'])
        
        
        for key2, value2 in grouped2:
            dictList3 = []
            anotherDict3 = {}
            context = key2
            anotherDict2['context'] =context
            grouped3 = df.loc[(df['title'] == title) & (df['context'] == context)].groupby(['id'])
            for key3, value3 in grouped3:
                n = grouped3.get_group(key3)
                dictList4 = []
                for m in n.index: 
                    anotherDict4 = {}
                    anotherDict4['answer_start'] = n.at[m, 'answer_start']
                    anotherDict4['text'] = n.at[m, 'text']
                    
                    dictList4.append(anotherDict4.copy())
                    
                anotherDict3['answers'] = dictList4
                curr_ques = value3["question"].reset_index()
                curr_id = value3["id"].reset_index()
                anotherDict3['question'] = curr_ques.iloc[0,1]
                anotherDict3['id'] = curr_id.iloc[0,1]
                dictList3.append(anotherDict3.copy())
            
                    
            anotherDict2['qas'] = dictList3
            dictList2.append(anotherDict2.copy())
                   
                    
        anotherDict1['paragraphs'] = dictList2
        dictList1.append(anotherDict1.copy())
    return(dictList1)
list1 = to_dictionary(df)
list1

[{'title': 'General',
  'paragraphs': [{'context': 'COVID 19 is the infectious disease caused by the most recently discovered coronavirus This new virus and disease were unknown before the outbreak began in Wuhan China in December 2019',
    'qas': [{'answers': [{'answer_start': array([156, 156]),
        'text': array(['Wuhan China in December 2019', 'Wuhan China in December 2019'],
              dtype=object)},
       {'answer_start': array([156, 156]),
        'text': array(['Wuhan China in December 2019', 'Wuhan China in December 2019'],
              dtype=object)}],
      'question': 'When does corona disease started',
      'id': 'KWCAH7Wqdk757HhaoSQVGV'},
     {'answers': [{'answer_start': array([0, 0]),
        'text': array(['COVID 19 is the infectious disease caused by the most recently discovered coronavirus',
               'COVID 19 is the infectious disease caused by the most recently discovered coronavirus'],
              dtype=object)},
       {'answer_start': array([

In [73]:
def convert_to_dictionary(df):
    group_by_title = df.groupby(['title'])
    final_dict = []
    for key1, value1 in group_by_title:
        temp_dict1 = {}
        dictList2 = []
        temp_dict2 = {}
        title = key1
        temp_dict1['title'] = title
        grouped2 = df.loc[df['title'] == title].groupby(['context'])
        for key2, value2 in grouped2:
            dictList3 = []
            temp_dict3 = {}
            context = key2
            temp_dict2['context'] =context
            grouped3 = df.loc[(df['title'] == title) & (df['context'] == context)].groupby(['id'])
            for key3, value3 in grouped3:
                n = grouped3.get_group(key3)
                dictList4 = []
                for m in n.index: 
                    temp_dict4 = {}
                    temp_dict4['answer_start'] = n.at[m, 'answer_start']
                    temp_dict4['text'] = n.at[m, 'text']
                    dictList4.append(temp_dict4.copy())
                temp_dict3['answers'] = dictList4
                curr_ques = value3["question"].reset_index()
                curr_id = value3["id"].reset_index()
                temp_dict3['question'] = curr_ques.iloc[0,1]
                temp_dict3['id'] = curr_id.iloc[0,1]
                dictList3.append(temp_dict3.copy()) 
            temp_dict2['qas'] = dictList3
            dictList2.append(temp_dict2.copy())
        temp_dict1['paragraphs'] = dictList2
        final_dict.append(temp_dict1.copy())
    return(final_dict)
squavd_list = convert_to_dictionary(df)

In [56]:
squavd_list

[{'title': 'General',
  'paragraphs': [{'context': 'COVID 19 is the infectious disease caused by the most recently discovered coronavirus This new virus and disease were unknown before the outbreak began in Wuhan China in December 2019',
    'qas': [{'answers': [{'answer_start': array([156, 156]),
        'text': array(['Wuhan China in December 2019', 'Wuhan China in December 2019'],
              dtype=object)},
       {'answer_start': array([156, 156]),
        'text': array(['Wuhan China in December 2019', 'Wuhan China in December 2019'],
              dtype=object)}],
      'question': 'When does corona disease started',
      'id': 'KWCAH7Wqdk757HhaoSQVGV'},
     {'answers': [{'answer_start': array([0, 0]),
        'text': array(['COVID 19 is the infectious disease caused by the most recently discovered coronavirus',
               'COVID 19 is the infectious disease caused by the most recently discovered coronavirus'],
              dtype=object)},
       {'answer_start': array([

In [65]:
def default(o):
    if isinstance(o, numpy.int64): return int(o)  
    raise TypeError
        
with open('dataset_json.json', 'w') as json_file:
    json_file.write(json.dumps({'data': list1,'version': '1.1'},default=default))
    json_file.close()

TypeError: 

In [57]:
with open('Test_Sample1.json', 'r') as f:
    train_data = json.load(f)

train_data = [item for topic in train_data['data'] for item in topic['paragraphs'] ]

In [9]:
train_data

[{'Context': '    Remember, a mask should only be used by health workers, care takers, and individuals with respiratory symptoms, such as fever and cough.\n    Before touching the mask, clean hands with an alcohol-based hand rub or soap and water\n    Take the mask and inspect it for tears or holes.\n    Orient which side is the top side (where the metal strip is).\n    Ensure the proper side of the mask faces outwards (the coloured side).\n    Place the mask to your face. Pinch the metal strip or stiff edge of the mask so it moulds to the shape of your nose.\n    Pull down the mask’s bottom so it covers your mouth and your chin.\n    After use, take off the mask; remove the elastic loops from behind the ears while keeping the mask away from your face and clothes, to avoid touching potentially contaminated surfaces of the mask.\n    Discard the mask in a closed bin immediately after use.\n    Perform hand hygiene after touching or discarding the mask – Use alcohol-based hand rub or, if

In [37]:
import pandas as pd
from ast import literal_eval

from cdqa.utils.filters import filter_paragraphs
from cdqa.utils.download import download_model, download_bnpp_data
from cdqa.pipeline.cdqa_sklearn import QAPipeline

# Download data and models
download_bnpp_data(dir='./data/bnpp_newsroom_v1.1/')
download_model(model='bert-squad_1.1', dir='./models')
download_model(model='distilbert-squad_1.1', dir='./models')


# Loading data and filtering / preprocessing the documents
df = pd.read_csv('df_corona.csv', converters={'paragraphs': literal_eval})
#df = filter_paragraphs(df)

df.paragraphs = df.paragraphs.astype(str)
df.dtypes

# Loading QAPipeline with CPU version of BERT Reader pretrained on SQuAD 1.1
cdqa_pipeline = QAPipeline(reader='models/bert_qa.joblib')

# Fitting the retriever to the list of documents in the dataframe
cdqa_pipeline.fit_retriever(df=df)

# Sending a question to the pipeline and getting prediction
query = 'Since when does the Excellence Program of BNP Paribas exist?'
prediction = cdqa_pipeline.predict(X=query)



bnpp_newsroom-v1.1.csv already downloaded

bert_qa.joblib already downloaded



ValueError: zero-dimensional arrays cannot be concatenated

In [19]:
df = pd.read_csv('corona.csv')

df['paragraphs'] = df[df.columns[1:]].apply(
    lambda x: x.dropna().values.tolist(),
    axis=1)

# Loading QAPipeline with CPU version of BERT Reader pretrained on SQuAD 1.1
cdqa_pipeline = QAPipeline(reader='models/bert_qa.joblib')

# Fitting the retriever to the list of documents in the dataframe
cdqa_pipeline.fit_retriever(df=df)

QAPipeline(reader=BertQA(adam_epsilon=1e-08, bert_model='bert-base-uncased',
                         do_lower_case=True, fp16=False,
                         gradient_accumulation_steps=1, learning_rate=5e-05,
                         local_rank=-1, loss_scale=0, max_answer_length=30,
                         n_best_size=20, no_cuda=False,
                         null_score_diff_threshold=0.0, num_train_epochs=3.0,
                         output_dir=None, predict_batch_size=8, seed=42,
                         server_ip='', server_po...size=8,
                         verbose_logging=False, version_2_with_negative=False,
                         warmup_proportion=0.1, warmup_steps=0),
           retrieve_by_doc=False,
           retriever=BM25Retriever(b=0.75, floor=None, k1=2.0, lowercase=True,
                                   max_df=0.85, min_df=2, ngram_range=(1, 2),
                                   preprocessor=None, stop_words='english',
                                   t

In [21]:
%%time
# Sending a question to the pipeline and getting prediction
query = 'Is it same as SARS'
prediction = cdqa_pipeline.predict(query=query)

print('query: {}\n'.format(query))
print('answer: {}\n'.format(prediction[0]))
print('title: {}\n'.format(prediction[1]))
print('paragraph: {}\n'.format(prediction[2]))

query: Is it same as SARS

answer: Severe Acute Respiratory Syndrome

title: What is a coronavirus

paragraph: In humans, several coronaviruses are known to cause respiratory infections ranging from the common cold to more severe diseases such as Middle East Respiratory Syndrome (MERS) and Severe Acute Respiratory Syndrome (SARS).

CPU times: user 12.3 s, sys: 1.04 s, total: 13.3 s
Wall time: 3.55 s


In [4]:
prediction

('Severe Acute Respiratory Syndrome',
 'What is a coronavirus',
 'In humans, several coronaviruses are known to cause respiratory infections ranging from the common cold to more severe diseases such as Middle East Respiratory Syndrome (MERS) and Severe Acute Respiratory Syndrome (SARS).',
 5.310092008113862)

In [5]:
df.sample(2)

,title,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,paragraphs
1,What is covid-19,COVID-19 is the infectious disease caused by t...,This new virus and disease were unknown before...,NaN,NaN,[COVID-19 is the infectious disease caused by ...
0,What is a coronavirus,Coronaviruses are a large family of viruses wh...,"In humans, several coronaviruses are known to ...","""The most recently discovered coronavirus cau...",NaN,[Coronaviruses are a large family of viruses w...


In [118]:
df = pd.read_csv('who.csv')
df = df.fillna(method='ffill')

df = df.replace({r'\s+$': '', r'^\s+': ''}, regex=True).replace(r'\n',  ' ', regex=True)
corona_data = pd.DataFrame()

In [119]:
corona_data[['date']] = df[['Last Retrieved Time']]
corona_data[['title']] = df[['Question']]
corona_data[['category']] = df[['Title']]
corona_data[['link']] = df[['Source URL']]
corona_data[['abstract']] = df[['Answer']]
corona_data[['paragraphs']] = df[['Context']]

corona_data['paragraphs'] = '[\'' + corona_data['paragraphs'].astype(str) + '\']'

# def literal_return(val):
#     try:
#         return ast.literal_eval(val)
#     except (ValueError, SyntaxError) as e:
#         return val

# from ast import literal_eval
# corona_data['paragraphs'] = corona_data.paragraphs.apply(lambda x: literal_eval(str(x)))

In [27]:
df = pd.read_csv('corona.csv')

df['paragraphs'] = df[df.columns[1:]].apply(
    lambda x: x.dropna().values.tolist(),
    axis=1)
#df['paragraphs'] = list(df['paragraphs'].values)
print(df.paragraphs)

0    [Coronaviruses are a large family of viruses w...
1    [COVID-19 is the infectious disease caused by ...
Name: paragraphs, dtype: object


In [28]:
from cdqa.pipeline import QAPipeline

cdqa_pipeline = QAPipeline(reader='models/bert_qa.joblib') # use 'distilbert_qa.joblib' for DistilBERT instead of BERT
cdqa_pipeline.fit_retriever(df=df)

QAPipeline(reader=BertQA(adam_epsilon=1e-08, bert_model='bert-base-uncased',
                         do_lower_case=True, fp16=False,
                         gradient_accumulation_steps=1, learning_rate=5e-05,
                         local_rank=-1, loss_scale=0, max_answer_length=30,
                         n_best_size=20, no_cuda=False,
                         null_score_diff_threshold=0.0, num_train_epochs=3.0,
                         output_dir=None, predict_batch_size=8, seed=42,
                         server_ip='', server_po...size=8,
                         verbose_logging=False, version_2_with_negative=False,
                         warmup_proportion=0.1, warmup_steps=0),
           retrieve_by_doc=False,
           retriever=BM25Retriever(b=0.75, floor=None, k1=2.0, lowercase=True,
                                   max_df=0.85, min_df=2, ngram_range=(1, 2),
                                   preprocessor=None, stop_words='english',
                                   t

In [29]:
query = "how corona virus spreads"
prediction = cdqa_pipeline.predict(query=query)

print('query: {}\n'.format(query))
print('answer: {}\n'.format(prediction[0]))
print('title: {}\n'.format(prediction[1]))
print('paragraph: {}\n'.format(prediction[2]))

query: how corona virus spreads

answer: several coronaviruses are known to cause respiratory infections

title: What is a coronavirus

paragraph: In humans, several coronaviruses are known to cause respiratory infections ranging from the common cold to more severe diseases such as Middle East Respiratory Syndrome (MERS) and Severe Acute Respiratory Syndrome (SARS).



In [68]:
df.sample(5)

,title,title,text,Source,Source URL,Organization,Country,Last Retrieved Time,answer_start,id,paragraphs
230,Info from Government,Is teleworking mandatory?,Teleworking is mandatory for all non-essential...,Official Website Belgium,https://www.info-coronavirus.be/en/faq/,Govt of Karnataka,India,2020-04-03 12:24:25.529410,0,jtUi4qUXhj9e5PyuwmnrS3,"[Is teleworking mandatory?, Teleworking is man..."
262,Info from Government,Can people who recover from COVID-19 be infect...,"We know that for similar coronaviruses, infect...",Official Website Belgium,https://www.info-coronavirus.be/en/faq/,Govt of Karnataka,India,2020-04-03 12:24:25.529410,0,jXpwh3Jacm3hmRmjMYTCYb,[Can people who recover from COVID-19 be infec...
32,Info from Government,Do persons suffering from pollen allergy or al...,A large proportion of the population (up to 15...,European Centre for Disease Prevention and Con...,https://www.ecdc.europa.eu/en/covid-19/questio...,Govt of Karnataka,India,2020-04-03 12:24:25.529410,0,5kviVF854TTPdYkoLy8oWk,[Do persons suffering from pollen allergy or a...
3,Info from Government,Can I catch COVID-19 from the feces of someone...,The risk of catching COVID-19 from the feces o...,Govt of Karnataka Website,https://www.mohfw.gov.in/pdf/FAQ.pdf,Govt of Karnataka,India,2020-04-03 12:24:25.529410,0,7ktc27267iizYbvrWjHYMB,[Can I catch COVID-19 from the feces of someon...
224,Info from Government,Some people have family living abroad and want...,"If possible, see if there is an alternative so...",Official Website Belgium,https://www.info-coronavirus.be/en/faq/,Govt of Karnataka,India,2020-04-03 12:24:25.529410,0,gLQsDn8EJF6fQGFTe7vJvG,[Some people have family living abroad and wan...


In [38]:
cdqa_pipeline = QAPipeline(reader='models/distilbert_qa.joblib') # use 'distilbert_qa.joblib' for DistilBERT instead of BERT
cdqa_pipeline.fit_reader('Test_Sample1.json')
cdqa_pipeline.dump_reader('path-to-save-bert-reader.joblib')

In [67]:
df = df[['context', 'question']]
df = df.fillna(method='ffill')

df['paragraphs'] = df[df.columns[1:]].apply(
    lambda x: x.dropna().values.tolist(),
    axis=1)

df.rename(columns={"question": "title"}, inplace=True)
df.drop(columns='context', inplace=True)
df.to_csv('df_corona.csv')
df.sample(5)

KeyError: "None of [Index(['context', 'question'], dtype='object')] are in the [columns]"

In [41]:
cdqa_pipeline = QAPipeline(reader='path-to-save-bert-reader.joblib')
cdqa_pipeline.fit_retriever(df=df)

QAPipeline(reader=BertQA(adam_epsilon=1e-08,
                         bert_model='distilbert-base-uncased',
                         do_lower_case=True, fp16=False,
                         gradient_accumulation_steps=1, learning_rate=5e-05,
                         local_rank=-1, loss_scale=0, max_answer_length=30,
                         n_best_size=20, no_cuda=False,
                         null_score_diff_threshold=0.0, num_train_epochs=3.0,
                         output_dir=None, predict_batch_size=8, seed=42,
                         server_ip='', ser...size=8,
                         verbose_logging=False, version_2_with_negative=False,
                         warmup_proportion=0.1, warmup_steps=0),
           retrieve_by_doc=False,
           retriever=BM25Retriever(b=0.75, floor=None, k1=2.0, lowercase=True,
                                   max_df=0.85, min_df=2, ngram_range=(1, 2),
                                   preprocessor=None, stop_words='english',
           

In [44]:
%%time
query = "what is coronna virus"
prediction = cdqa_pipeline.predict(query=query)

print('query: {}\n'.format(query))
print('answer: {}\n'.format(prediction[0]))
print('title: {}\n'.format(prediction[1]))
print('paragraph: {}\n'.format(prediction[2]))

query: what is coronna virus

answer: The risk depends on where you are - and more specifically, whether there is a COVID-19 outbreak unfolding there

title: How likely am I to catch covid-19

paragraph: The risk depends on where you  are - and more specifically, whether there is a COVID-19 outbreak unfolding there.

For most people in most locations the risk of catching COVID-19 is still low. However, there are now places around the world (cities or areas) where the disease is spreading. For people living in, or visiting, these areas the risk of catching COVID-19 is higher. Governments and health authorities are taking vigorous action every time a new case of COVID-19 is identified. Be sure to comply with any local restrictions on travel, movement or large gatherings. Cooperating with disease control efforts will reduce your risk of catching or spreading COVID-19.

COVID-19 outbreaks can be contained and transmission stopped, as has been shown in China and some other countries. Unfort

In [80]:
import pandas as pd
from ast import literal_eval

from cdqa.utils.filters import filter_paragraphs
from cdqa.utils.download import download_model, download_bnpp_data
from cdqa.pipeline.cdqa_sklearn import QAPipeline

#read the cleaned dataset and just take question and context for our model
df = pd.read_csv('data/dataset_collected.csv', usecols=['question', 'context'])

#convert paragraphs to a list
df['paragraphs'] = df[df.columns[1:]].apply(
    lambda x: x.dropna().values.tolist(),
    axis=1)

df.rename(columns={"question": "title"}, inplace=True)
df.drop(columns='context', inplace=True)
df.to_csv('df_corona.csv', index=False)
print(df.sample(5))

                                                 title  \
232  Are photographers allowed to do assignments ou...   
297  Are specific measures applicable to the transp...   
116          Are some people more at risk than others?   
65   How is an infection with the new coronavirus d...   
356  Can humans become infected with the COVID-19 f...   

                                            paragraphs  
232                                  [No, they can’t.]  
297  [The most important thing is that no new socia...  
116  [Elderly people above 70 years of age and thos...  
65   [Yes. The likelihood of an infected person con...  
356  [Possible animal sources of COVID-19 have not ...  


In [82]:
#use a lighter pipleline model to build pipeline on top of it
cdqa_pipeline = QAPipeline(reader='models/distilbert_qa.joblib')
cdqa_pipeline.fit_retriever(df=df)

QAPipeline(reader=BertQA(adam_epsilon=1e-08,
                         bert_model='distilbert-base-uncased',
                         do_lower_case=True, fp16=False,
                         gradient_accumulation_steps=1, learning_rate=5e-05,
                         local_rank=-1, loss_scale=0, max_answer_length=30,
                         n_best_size=20, no_cuda=False,
                         null_score_diff_threshold=0.0, num_train_epochs=3.0,
                         output_dir=None, predict_batch_size=8, seed=42,
                         server_ip='', ser...size=8,
                         verbose_logging=False, version_2_with_negative=False,
                         warmup_proportion=0.1, warmup_steps=0),
           retrieve_by_doc=False,
           retriever=BM25Retriever(b=0.75, floor=None, k1=2.0, lowercase=True,
                                   max_df=0.85, min_df=2, ngram_range=(1, 2),
                                   preprocessor=None, stop_words='english',
           

In [86]:
%%time
query = "Can I travel ?"
prediction = cdqa_pipeline.predict(query=query)

print('Query : {}\n'.format(query))
print('Answer from Bot: {}\n'.format(prediction[0]))
print('Matched to Question : {}\n'.format(prediction[1]))
print('Paragraph pickup from : {}\n'.format(prediction[2]))

Query : Can I travel ?

Answer from Bot: Any non-essential travel abroad is prohibited

Matched to Question : Is it still allowed to travel abroad?

Paragraph pickup from : Any non-essential travel abroad is prohibited. The following reasons in particular are considered to be essential reasons for travelling abroad:

CPU times: user 24.5 s, sys: 1.78 s, total: 26.3 s
Wall time: 7.06 s
